<a href="https://colab.research.google.com/github/Tanviwasule/Action-Recognition-with-Images/blob/main/Alexnet_of_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))


GPUs: []


In [ ]:
import zipfile
import os

zip_path = "/content/archive (7).zip"
extract_dir = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


for root, dirs, files in os.walk(extract_dir):
    level = root.replace(extract_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:
        print(f"{subindent}{f}")


dataset/
    README.md
    Animals/
        snakes/
            2_0964.jpg
            2_0202.jpg
            2_0341.jpg
            2_0362.jpg
            2_0056.jpg
        cats/
            0_0627.jpg
            0_0045.jpg
            0_0516.jpg
            0_0526.jpg
            0_0041.jpg
        dogs/
            1_0022.jpg
            1_0040.jpg
            1_0822.jpg
            1_0115.jpg
            1_0943.jpg


In [ ]:
data_dir = "/content/dataset/Animals"


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
import os
import numpy as np

data_dir = "/content/dataset/Animals"
img_size = (227, 227)
batch_size = 32
seed = 123
num_epochs = 30


np.random.seed(seed)
tf.random.set_seed(seed)


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="training",
    seed=seed
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    batch_size=batch_size,
    image_size=img_size,
    validation_split=0.2,
    subset="validation",
    seed=seed
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

def build_alexnet(input_shape=(227,227,3), num_classes=3, dropout_rate=0.5):
    model = models.Sequential(name="AlexNet_practical")
    model.add(layers.Input(shape=input_shape))


    model.add(layers.Rescaling(1.0/255))


    model.add(layers.RandomFlip("horizontal"))
    model.add(layers.RandomRotation(0.08))
    model.add(layers.RandomZoom(0.08))

    # Conv1
    model.add(layers.Conv2D(96, (11,11), strides=4, padding='valid', activation='relu', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((3,3), strides=2))

    # Conv2
    model.add(layers.Conv2D(256, (5,5), padding='same', activation='relu', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((3,3), strides=2))

    # Conv3, Conv4, Conv5
    model.add(layers.Conv2D(384, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(384, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((3,3), strides=2))

    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(dropout_rate))

    # Output
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

model = build_alexnet(input_shape=(*img_size, 3), num_classes=num_classes, dropout_rate=0.5)
model.summary()

opt = optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


checkpoint_cb = callbacks.ModelCheckpoint("alexnet_best.h5", save_best_only=True, monitor='val_accuracy', mode='max')
earlystop_cb = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr_cb = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

cb_list = [checkpoint_cb, earlystop_cb, reduce_lr_cb]

# ----- Train -----
history = model.fit(
    train_ds,
    epochs=num_epochs,
    validation_data=val_ds,
    callbacks=cb_list
)

# ----- Evaluate -----
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc*100:.2f}%")

# ----- Save final model -----
model.save("alexnet_final.h5")




Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 600 files for validation.
Classes: ['cats', 'dogs', 'snakes']


Model: "AlexNet_practical"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 227, 227, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_1 (RandomFlip)      │ (None, 227, 227, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 227, 227, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_1 (RandomZoom)      │ (None, 227, 227, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 55, 55, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 27, 27, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 13, 13, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4096)           │             

 Total params: 58,299,139 (222.39 MB)

 Trainable params: 58,296,387 (222.38 MB)

...

75/75 ━━━━━━━━━━━━━━━━━━━━ 478s 6s/step - accuracy: 0.4989 - loss: 6.0201 - val_accuracy: 0.4633 - val_loss: 2.8821 - learning_rate: 0.0100
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5244 - loss: 1.8657

75/75 ━━━━━━━━━━━━━━━━━━━━ 510s 7s/step - accuracy: 0.5246 - loss: 1.8618 - val_accuracy: 0.4767 - val_loss: 1.1727 - learning_rate: 0.0100
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5831 - loss: 1.0325

75/75 ━━━━━━━━━━━━━━━━━━━━ 506s 7s/step - accuracy: 0.5830 - loss: 1.0317 - val_accuracy: 0.5933 - val_loss: 0.8404 - learning_rate: 0.0100
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 478s 6s/step - accuracy: 0.6108 - loss: 0.8156 - val_accuracy: 0.4800 - val_loss: 1.2202 - learning_rate: 0.0100
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6046 - loss: 0.8037

75/75 ━━━━━━━━━━━━━━━━━━━━ 501s 7s/step - accuracy: 0.6046 - loss: 0.8037 - val_accuracy: 0.6267 - val_loss: 0.7522 - learning_rate: 0.0100
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6262 - loss: 0.7829

75/75 ━━━━━━━━━━━━━━━━━━━━ 502s 7s/step - accuracy: 0.6262 - loss: 0.7828 - val_accuracy: 0.6383 - val_loss: 0.7522 - learning_rate: 0.0100
Epoch 8/30
19/75 ━━━━━━━━━━━━━━━━━━━━ 5:27 6s/step - accuracy: 0.5940 - loss: 0.7717